In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy 
from collections import OrderedDict
from tqdm import tqdm
import pandas as pd
%matplotlib inline

In [ ]:
# !unzip 'Assignment 5 Dataset.zip'

In [ ]:
# roll number
roll_num = 69
batch_size = 64
epochs = 10

## Loading Data

In [ ]:
data_dir = 'Assignment 5 Dataset'

# Defining transformations
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


In [ ]:
# Applying transformations on the data
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/validation', transform=valid_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)


# Data Loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)


print("Classes: ")
class_names = train_data.classes
print(class_names)

## Showing data

In [ ]:
# Showing Images
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(trainloader))
show_databatch(inputs, classes)

## TASK 1

### Loading Models

In [ ]:
# Load the pretrained models from pytorch
vgg16 = models.vgg16(pretrained=True)

# resnet18 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
resnet18 = models.resnet18(pretrained=True)

In [ ]:
# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

for param in resnet18.parameters():
    param.requires_grad = False


### Removing and Adding New Layers

In [ ]:
# Removing Last Line
num_features = vgg16.classifier[0].in_features
features = list(vgg16.classifier.children())[0:-7] # Remove last layer

# resnet18 = nn.Sequential(*list(resnet18.modules())[:-1]) 


In [ ]:
new_feature = ( roll_num * 10 ) + 100
# Adding Layers
# vgg16
features.extend([nn.Linear(num_features, new_feature)])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature)])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, len(class_names) )])
vgg16.classifier = nn.Sequential(*features)

# resNet18
resnet18.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(512, new_feature)), ('relu', nn.ReLU()), ('fc2', nn.Linear(new_feature, 2)), ('output', nn.LogSoftmax(dim=1))]))


In [ ]:
# Printing
print(vgg16)
print('---------------')
print(resnet18)

In [ ]:
# Optimizer
criterion = nn.CrossEntropyLoss()
vgg16_optimizer = optim.SGD(vgg16.parameters(), lr=0.0001, momentum=0.9)
resnet18_optimizer = optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
# Setting to either GPU or CPU based on availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

### VGG-16

In [ ]:
# Training VGG-16
vgg16_cross_entropy = []
vgg16_valid_accuracy = []

valid_normal_true = 0
valid_normal_false = 0
valid_infected_true = 0
valid_infected_false  = 0


# Running Epochs
for epoch in range(epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        vgg16_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        vgg16_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    vgg16_cross_entropy.append(loss.data)
        
    torch.save(vgg16.state_dict(), 'vgg16_FC_Only.pth')
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for l, p in zip(labels, predicted):
                if(l.item() == 0):
                    if(p.item() == 0):
                        valid_normal_true +=1
                    else:
                        valid_normal_false +=1
                else:
                    if(p.item() == 1):
                        valid_infected_true +=1
                    else:
                        valid_infected_false +=1
    vgg16_valid_accuracy.append((100 * correct / total))

    
print('VGG-16 Trained!')

In [ ]:
# Drawing Confusion Table for VGG-16
data = np.array([[valid_normal_true, valid_normal_false],
                 [valid_infected_false,valid_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Drawing Accuracy of VGG-16
print('*** Validation Accuracy ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), vgg16_valid_accuracy, color='green')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()    


print('*** Cross Entropy Curve ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), vgg16_cross_entropy, color='red')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy')
plt.show() 

In [ ]:
# Runing VGG-16 on Test Data
test_normal_true = 0
test_normal_false = 0
test_infected_true = 0
test_infected_false = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for l, p in zip(labels, predicted):
            if(l.item() == 0):
                if(p.item() == 0):
                    test_normal_true +=1
                else:
                    test_normal_false +=1
            else:
                if(p.item() == 1):
                    test_infected_true +=1
                else:
                    test_infected_false +=1
    print('Accuracy of the test Data : %d %%' % (100 * correct / total))

In [ ]:
# Drawing Confusion Table for VGG-16 Test data
data = np.array([[test_normal_true, test_normal_false],
                 [test_infected_false,test_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Reacall Precision and F_Score for Tesing Data of VGG-16
test_recall = 0
test_precision = 0 
test_f_score = 0

test_recall = test_normal_true/(test_normal_true+test_normal_false)
test_precision = test_normal_true/(test_normal_true+valid_normal_false)
test_f_score = 2*(test_precision*test_recall)/(test_precision+test_recall)
print('VGG-16 Valid F1 Score: %d %%' % (100 * test_f_score)) 

### ResNet-18

In [ ]:
# Training RexNet-18
resnet18_cross_entropy = []
resnet18_valid_accuracy = []

valid_normal_true = 0
valid_normal_false = 0
valid_infected_true = 0
valid_infected_false  = 0


# Running Epochs
for epoch in range(epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        resnet18_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        resnet18_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    resnet18_cross_entropy.append(loss.data)
        
    torch.save(resnet18.state_dict(), 'res18_FC_Only.pth')
     
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for l, p in zip(labels, predicted):
                if(l.item() == 0):
                    if(p.item() == 0):
                        valid_normal_true +=1
                    else:
                        valid_normal_false +=1
                else:
                    if(p.item() == 1):
                        valid_infected_true +=1
                    else:
                        valid_infected_false +=1
 
    resnet18_valid_accuracy.append((100 * correct / total))
    

print('ResNet-18 Trained!')

In [ ]:
# Drawing Confusion Table for ResNet-18
data = np.array([[valid_normal_true, valid_normal_false],
                 [valid_infected_false,valid_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Drawing Accuracy of ResNet-18
print('*** Validation Accuracy ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), resnet18_valid_accuracy, color='green')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()    


print('*** Cross Entropy Curve ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), resnet18_cross_entropy, color='red')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy')
plt.show() 

In [ ]:
# Runing ResNet-18 on Testing Data
test_normal_true = 0
test_normal_false = 0
test_infected_true = 0
test_infected_false = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for l, p in zip(labels, predicted):
            if(l.item() == 0):
                if(p.item() == 0):
                    test_normal_true +=1
                else:
                    test_normal_false +=1
            else:
                if(p.item() == 1):
                    test_infected_true +=1
                else:
                    test_infected_false +=1
    print('Accuracy of the test Data : %d %%' % (100 * correct / total))

In [ ]:
# Drawing Confusion Table for ResNet-18 Test data
data = np.array([[test_normal_true, test_normal_false],
                 [test_infected_false,test_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Reacall Precision and F_Score for Tesing Data of ResNet-18
test_recall = 0
test_precision = 0 
test_f_score = 0

test_recall = test_normal_true/(test_normal_true+test_normal_false)
test_precision = test_normal_true/(test_normal_true+valid_normal_false)
test_f_score = 2*(test_precision*test_recall)/(test_precision+test_recall)
print('ResNet test F1 Score: %d %%' % (100 * test_f_score))

## TASK 2 

### Freezing Few Layers

#### Initialzing Models

In [ ]:
# Initialize Network
vgg16 = models.vgg16(pretrained=True)
resnet18 = models.resnet18(pretrained=True)

#### Freezing Layers

In [ ]:
# Feezing only even layers

# count = 0
# for param in vgg16.features:
#     print(count,param)
#     count +=1
    
count = 0

for param in vgg16.features.parameters():
    if count == 0 or count == 2 or count == 5 or count == 10 or count == 14 or count ==21 or count == 26:
        param.require_grad = False
    count += 1
count = 0

for param in resnet18.parameters():
    if count%4 == 0:
        param.require_grad = False
    count +=1

#### Removing and Adding Layers

In [ ]:
# Removing Last Line

num_features = vgg16.classifier[0].in_features
features = list(vgg16.classifier.children())[:-7] # Remove last layer

# resnet18 = nn.Sequential(*list(resnet18.modules())[:-1]) 

In [ ]:
new_feature = ( roll_num * 10 ) + 100
# Adding Layers

features.extend([nn.Linear(num_features, new_feature)])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature)])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, len(class_names) )])
vgg16.classifier = nn.Sequential(*features)

# resNet18
resnet18.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(512, new_feature)), ('relu', nn.ReLU()), ('fc2', nn.Linear(new_feature, 2)), ('output', nn.LogSoftmax(dim=1))]))


In [ ]:
# Optimizer
criterion = nn.CrossEntropyLoss()
vgg16_optimizer = optim.SGD(vgg16.parameters(), lr=0.0001, momentum=0.9)
resnet18_optimizer = optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
# Setting to either GPU or CPU based on availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

### VGG-16

In [ ]:
# training VGG-16
vgg16_cross_entropy = []
vgg16_valid_accuracy = []

valid_normal_true = 0
valid_normal_false = 0
valid_infected_true = 0
valid_infected_false  = 0



# Running Epochs
for epoch in range(epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        vgg16_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        vgg16_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    vgg16_cross_entropy.append(loss.data)
        
#     torch.save(vgg16.state_dict(), 'vgg16_FC_Only.pth')
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for l, p in zip(labels, predicted):
                if(l.item() == 0):
                    if(p.item() == 0):
                        valid_normal_true +=1
                    else:
                        valid_normal_false +=1
                else:
                    if(p.item() == 1):
                        valid_infected_true +=1
                    else:
                        valid_infected_false +=1
    vgg16_valid_accuracy.append((100 * correct / total))

print('VGG-16 Trained!')

In [ ]:
# drawing Confusion Table for VGG-16 Some Layers Freeze
data = np.array([[valid_normal_true, valid_normal_false],
                 [valid_infected_false,valid_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Drawing Accuracy of VGG-16 Some Layers Freeze
print('*** Validation Accuracy ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), vgg16_valid_accuracy, color='green')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()    


print('*** Cross Entropy Curve ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), vgg16_cross_entropy, color='red')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy')
plt.show() 

In [ ]:
# Runing VGG-16 on Testing Data
test_normal_true = 0
test_normal_false = 0
test_infected_true = 0
test_infected_false = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for l, p in zip(labels, predicted):
            if(l.item() == 0):
                if(p.item() == 0):
                    test_normal_true +=1
                else:
                    test_normal_false +=1
            else:
                if(p.item() == 1):
                    test_infected_true +=1
                else:
                    test_infected_false +=1
    print('Accuracy of VGG-16 model on the test Data : %d %%' % (100 * correct / total))

In [ ]:
# Drawing Confusion Table for VGG-16 
data = np.array([[test_normal_true, test_normal_false],
                 [test_infected_false, test_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Reacall Precision and F_Score for Tesing Data of VGG-16
test_recall = 0
test_precision = 0 
test_f_score = 0

test_recall = test_normal_true/(test_normal_true+test_normal_false)
test_precision = test_normal_true/(test_normal_true+valid_normal_false)
test_f_score = 2*(test_precision*test_recall)/(test_precision+test_recall)
print('VGG-16 Valid F1 Score: %d %%' % (100 * test_f_score)) 

### ResNet-18

In [ ]:
# Training ResNet-18
resnet18_cross_entropy = []
resnet18_valid_accuracy = []
resnet18_train_accuracy = []
valid_normal_true = 0
valid_normal_false = 0
valid_infected_true = 0
valid_infected_false  = 0

# Running Epochs
for epoch in range(epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        resnet18_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        resnet18_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    resnet18_cross_entropy.append(loss.data)
        
#     torch.save(resnet18.state_dict(), 'res18_FC_Only.pth')
     
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for l, p in zip(labels, predicted):
                if(l.item() == 0):
                    if(p.item() == 0):
                        valid_normal_true +=1
                    else:
                        valid_normal_false +=1
                else:
                    if(p.item() == 1):
                        valid_infected_true +=1
                    else:
                        valid_infected_false +=1
    resnet18_train_accuracy.append((100 * correct / total))

    
    
print('ResNet-18 Trained!')

In [ ]:
# Drawing Confusion Table for ResNet-18
data = np.array([[valid_normal_true, valid_normal_false],
                 [valid_infected_false,valid_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Drawing Accuracy of ResNet-18
print('*** Validation Accuracy ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), resnet18_train_accuracy, color='green')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()    


print('*** Cross Entropy Curve ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), resnet18_cross_entropy, color='red')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy')
plt.show() 

In [ ]:
# Runing ResNet-18 on Testing Data
test_normal_true = 0
test_normal_false = 0
test_infected_true = 0
test_infected_false = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for l, p in zip(labels, predicted):
            if(l.item() == 0):
                if(p.item() == 0):
                    test_normal_true +=1
                else:
                    test_normal_false +=1
            else:
                if(p.item() == 1):
                    test_infected_true +=1
                else:
                    test_infected_false +=1
    print('Accuracy of ResNet Model on the test Data : %d %%' % (100 * correct / total))

In [ ]:
# Reacall Precision and F_Score for Tesing Data of ResNet-18
test_recall = 0
test_precision = 0 
test_f_score = 0

test_recall = test_normal_true/(test_normal_true+test_normal_false)
test_precision = test_normal_true/(test_normal_true+valid_normal_false)
test_f_score = 2*(test_precision*test_recall)/(test_precision+test_recall)
print('ResNet-18 Valid F1 Score: %d %%' % (100 * test_f_score)) 

### UnFreezing All Layers

### VGG-16

#### Initializing

In [ ]:
# Initialize Network

vgg16 = models.vgg16(pretrained=True)
resnet18 = models.resnet18(pretrained=True)

#### Adding and Removing Layers

In [ ]:
# Removing Last Line

num_features = vgg16.classifier[0].in_features
features = list(vgg16.classifier.children())[0:-7] # Remove last layer

# resnet18 = nn.Sequential(*list(resnet18.modules())[:-1]) 

In [ ]:
new_feature = ( roll_num * 10 ) + 100
# Adding Layers
# vgg16
features.extend([nn.Linear(num_features, new_feature)])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature )])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, new_feature)])
features.extend([nn.ReLU(inplace=True)])
features.extend([nn.Dropout(p=0.5, inplace=False)])
features.extend([nn.Linear( new_feature, len(class_names) )])
vgg16.classifier = nn.Sequential(*features)

# resNet18
resnet18.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(512, new_feature)), ('relu', nn.ReLU()), ('fc2', nn.Linear(new_feature, 2)), ('output', nn.LogSoftmax(dim=1))]))


In [ ]:
# Optimizer
criterion = nn.CrossEntropyLoss()
vgg16_optimizer = optim.SGD(vgg16.parameters(), lr=0.0001, momentum=0.9)
resnet18_optimizer = optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
# Setting to either GPU or CPU based on availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

In [ ]:
# training VGG-16
vgg16_cross_entropy = []
vgg16_valid_accuracy = []

valid_normal_true = 0
valid_normal_false = 0
valid_infected_true = 0
valid_infected_false  = 0

# Running Epochs
for epoch in range(epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        vgg16_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        vgg16_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    vgg16_cross_entropy.append(loss.data)
        
    torch.save(vgg16.state_dict(), 'vgg16_FC_Entire.pth')
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for l, p in zip(labels, predicted):
                if(l.item() == 0):
                    if(p.item() == 0):
                        valid_normal_true +=1
                    else:
                        valid_normal_false +=1
                else:
                    if(p.item() == 1):
                        valid_infected_true +=1
                    else:
                        valid_infected_false +=1
    vgg16_valid_accuracy.append((100 * correct / total))

    
print('VGG-16 Trained!')

In [ ]:
# Drawing Confusion Table for VGG-16
data = np.array([[valid_normal_true, valid_normal_false],
                 [valid_infected_false,valid_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Drawing Accuracy of VGG-16
print('*** Validation Accuracy ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), vgg16_valid_accuracy, color='green')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()    


print('*** Cross Entropy Curve ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), vgg16_cross_entropy, color='red')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy')
plt.show() 

In [ ]:
# Runing VGG-18 on Testing Data
test_normal_true = 0
test_normal_false = 0
test_infected_true = 0
test_infected_false = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for l, p in zip(labels, predicted):
            if(l.item() == 0):
                if(p.item() == 0):
                    test_normal_true +=1
                else:
                    test_normal_false +=1
            else:
                if(p.item() == 1):
                    test_infected_true +=1
                else:
                    test_infected_false +=1
                    
    print('Accuracy of VGG-16 Model on the test Data : %d %%' % (100 * correct / total))

In [ ]:
# drawing Confusion Table for VGG Test data
data = np.array([[test_normal_true, test_normal_false],
                 [test_infected_false,test_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# F1 Score
precision = test_normal_true/(test_normal_true+test_normal_false)
recall = test_normal_true/(test_normal_true+test_infected_false)
f_score = 2*(precision*recall)/(precision+recall)
print('VGG Test F1 Score: %d %%' % (100 * f_score))  

### ResNet-18

In [ ]:
# Traing ResNet-18
resnet18_cross_entropy = []
resnet18_valid_accuracy = []

valid_normal_true = 0
valid_normal_false = 0
valid_infected_true = 0
valid_infected_false  = 0

# Running Epochs
for epoch in range(epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        resnet18_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        resnet18_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    resnet18_cross_entropy.append(loss.data)
        
    torch.save(resnet18.state_dict(), 'res18_FC_Only.pth')
     
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for l, p in zip(labels, predicted):
                if(l.item() == 0):
                    if(p.item() == 0):
                        valid_normal_true +=1
                    else:
                        valid_normal_false +=1
                else:
                    if(p.item() == 1):
                        valid_infected_true +=1
                    else:
                        valid_infected_false +=1
    resnet18_valid_accuracy.append((100 * correct / total))

    
print('ResNet-18 Trained!')

In [ ]:
# Drawing Confusion Table for ResNet-18
data = np.array([[valid_normal_true, valid_normal_false],
                 [valid_infected_false,valid_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
# Drawing Accuracy of ResNet-18
print('*** Validation Accuracy ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), resnet18_valid_accuracy, color='green')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()    


print('*** Cross Entropy Curve ***')
plt.figure(figsize=(10,5))
plt.plot(range(epochs), resnet18_cross_entropy, color='red')
plt.xlabel('Epochs')
plt.ylabel('Cross Entropy')
plt.show()

In [ ]:
# Runing ResNet-18 on Testing Data
test_normal_true = 0
test_normal_false = 0
test_infected_true = 0
test_infected_false = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for l, p in zip(labels, predicted):
            if(l.item() == 0):
                if(p.item() == 0):
                    test_normal_true +=1
                else:
                    test_normal_false +=1
            else:
                if(p.item() == 1):
                    test_infected_true +=1
                else:
                    test_infected_false +=1
    print('Accuracy of ResNet Model on the test Data : %d %%' % (100 * correct / total))

In [ ]:
# F1 Score
precision = test_normal_true/(test_normal_true+test_normal_false)
recall = test_normal_true/(test_normal_true+test_infected_false)
f_score = 2*(precision*recall)/(precision+recall)
print('ResNet Test F1 Score: %d %%' % (100 * f_score)) 

In [ ]:
# Drawing Confusion Table for ResNet Test data
data = np.array([[test_normal_true, test_normal_false],
                 [test_infected_false,test_infected_true, ]])
print("                Predicted Normal      Pridicted False ")

row_format ="{:>15}" * (len(class_names) + 1)
print(row_format.format("", *class_names))
for team, row in zip(class_names, data):
    print( row_format.format(team, *row))

In [ ]:
import torch
torch.cuda.empty_cache()